In [57]:
import os
import boto3
import requests
import pandas as pd
import pyarrow as pa
from io import BytesIO
import pyarrow.parquet as pq
from datetime import datetime

In [58]:
API_LINK = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [59]:
requisicao = requests.get(API_LINK)

In [60]:
wheater_json = requisicao.json()

In [61]:
weather_df = pd.json_normalize(wheater_json)

In [62]:
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1703383315,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1008,1.49,124,5.4,62,2,2020383,BR,1703319714,1703370296


In [63]:
df_lon_lat = pd.DataFrame(wheater_json['coord'], index=[0])

In [64]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [65]:
df_weather = pd.DataFrame(wheater_json['weather'], index=[0])

In [66]:
df_weather

,id,main,description,icon
0,803,Clouds,broken clouds,04n


In [67]:
df_main = pd.DataFrame(wheater_json['main'], index=[0])

In [68]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,297.12,297.82,296.28,298.57,1012,86,1012,1008


In [69]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [70]:
weather_df = pd.DataFrame(weather_df)

In [71]:
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1703383315,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1008,1.49,124,5.4,62,2,2020383,BR,1703319714,1703370296


In [72]:
df_clumns_selected = weather_df[selected_columns]

In [73]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1703383315,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,297.12,...,1008,1.49,124,5.4,62,2,2020383,BR,1703319714,1703370296


In [74]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [75]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,803.0,Clouds,broken clouds,04n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,297.12,297.82,296.28,298.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1008.0,1.49,124.0,5.4,62.0,2.0,2020383.0,BR,1.703320e+09,1.703370e+09


In [76]:
super_df.to_parquet("C:/Users/crist/Desktop/s3_testes/super_df.parquet")

Save at S3

In [77]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [78]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [79]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [80]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [81]:
dia_da_exucucao

'2023-12-23-23-01-47'

In [82]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [83]:
s3_file_name

'df_teste_2023-12-23-23-01-47.parquet'

In [84]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

---------------------------------

CATUABA